In [1]:
# Correlation matrices saved as Excel file
# interaction map 8d, bins, shape modes
# concordance 8d, bins, shape modes
# concordance edge, non-edge, i1, m1, i2, m2 

In [2]:
!pwd
!date

/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_notebooks/Tables
Mon May 30 20:14:53 PDT 2022


In [3]:
import os
import sys
import importlib
import concurrent
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
from skimage import io as skio
import matplotlib.pyplot as plt
from aicscytoparam import cytoparam
from aicsshparam import shtools, shparam
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from cvapipe_analysis.tools import io, viz, general, controller, shapespace, plotting

sys.path.insert(1, '../tools')
import common

In [4]:
path_config = Path("/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/")
config = general.load_config_file(path_config)
control = controller.Controller(config)
device = io.LocalStagingIO(control)
print(control.get_staging())

/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_variance


In [5]:
variables = control.get_variables_values_for_aggregation()
df_agg = shapespace.ShapeSpaceBasic.get_aggregated_df(variables)
variables.update({"shape_mode": ["NdSphere"], "mpId": [control.get_center_map_point_index()]})
df_sphere = shapespace.ShapeSpaceBasic.get_aggregated_df(variables)
df_agg = df_agg.append(df_sphere, ignore_index=True)
df_agg =  df_agg.drop(columns=["structure"]).drop_duplicates().reset_index(drop=True)

# Avg. spatial interactions

In [18]:
full = np.zeros((8*25, 9*25))

In [27]:
for smid, sm in enumerate(control.get_shape_modes()):
    for mpId in control.get_map_point_indexes():
        row = df_agg.loc[(df_agg.mpId==mpId)&(df_agg.shape_mode==sm)]
        row = row.loc[row.index[0]]
        corr = device.build_correlation_matrix_of_avg_reps_from_corr_values(row)
        full[(smid*25):((smid+1)*25), ((mpId-1)*25):(mpId*25)] = corr

In [28]:
pd.DataFrame(full).to_csv("avg_spatial_int.csv")

In [8]:
row = df_agg.loc[df_agg.index[-1]]
corr = device.build_correlation_matrix_of_avg_reps_from_corr_values(row)
corr = np.array(corr)
pd.DataFrame(corr).to_csv("avg_spatial_int_8d.csv")

# Concordance

In [44]:
full = np.zeros((8*25, 9*25))

In [45]:
for smid, sm in enumerate(control.get_shape_modes()):
    for mpId in control.get_map_point_indexes():
        row = df_agg.loc[(df_agg.mpId==mpId)&(df_agg.shape_mode==sm)]
        row = row.loc[row.index[0]]
        corr = device.read_corelation_matrix(row)
        corr = plotting.PlotMaker.get_aggregated_matrix_from_df(control.get_gene_names(), corr)
        full[(smid*25):((smid+1)*25), ((mpId-1)*25):(mpId*25)] = corr

In [46]:
pd.DataFrame(full).to_csv("concordance.csv", index=False, header=False)

In [42]:
row = df_agg.loc[df_agg.index[-1]]
corr = device.read_corelation_matrix(row)
corr = plotting.PlotMaker.get_aggregated_matrix_from_df(control.get_gene_names(), corr)
pd.DataFrame(corr).to_csv("concordance_8d.csv", index=False, header=False)

# Concordance for subsets

In [60]:
for ds, staging in zip(["non-edge","i1","i2","edge","m1","m2"],["local_staging_variance_edges","local_staging_variance_m1m2","local_staging_variance_m3","local_staging_edge_cells_midpoint_expanded","local_staging_m1m2","local_staging_m3"]):
    path_step = Path(f"/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/{staging}/correlation")
    ctrl, dev = common.get_managers_from_step_path(path_step)
    variables = ctrl.get_variables_values_for_aggregation()
    df_agg = shapespace.ShapeSpaceBasic.get_aggregated_df(variables)
    df_agg =  df_agg.drop(columns=["structure"]).drop_duplicates().reset_index(drop=True)
    corr = dev.read_corelation_matrix(df_agg.loc[0])
    print(ds, corr.shape)
    corr = plotting.PlotMaker.get_aggregated_matrix_from_df(control.get_gene_names(), corr)
    pd.DataFrame(corr).to_csv(f"concordance_{ds}.csv", index=False, header=False)

non-edge (4477, 4477)


/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/cvapipe_analysis/tools/plotting.py:124: PerformanceWarning: indexing past lexsort depth may impact performance.
  values = df_corr.loc[(gene1, gene2)].values


i1 (1969, 1969)
i2 (877, 877)
edge (5169, 5169)
m1 (2201, 2201)
m2 (981, 981)


In [61]:
common.now("complete.")

complete. 2022-05-31 22:15:44
